# 1NN vs Random Forest
В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [20]:
from sklearn import datasets

In [21]:
digits = datasets.load_digits()

In [22]:
X, y = digits.data, digits.target

In [23]:
X_train, X_test = X[:len(X) * 3 // 4, :], X[len(X) * 3 // 4:, :]
y_train, y_test = y[:len(y) * 3 // 4], y[len(y) * 3 // 4:]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1347, 64) (1347,) (450, 64) (450,)


Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [54]:
import numpy as np
def find_neigbour(X, p):
    best_x = [np.inf, 0]
    for idx, x in enumerate(X):
        if (best_x[0] >= np.linalg.norm(p - x)):
            best_x[0] = np.linalg.norm(p - x)
            best_x[1] = idx
    return best_x[1]

def get_1NN_error_rate(X_train, y_train, X_test, y_test):
    erros = 0
    for p, y_true in zip(X_test, y_test):
        find_idx = find_neigbour(X_train, p)
        y_pred = y_train[find_idx]
        erros += y_true != y_pred
        if (y_true != y_pred):
            print(y_true, y_pred)
    erros_rate = erros / X_test.shape[0]
    return erros_rate

In [55]:
error_rate_1NN = get_1NN_error_rate(X_train, y_train, X_test, y_test)

5 6
8 9
8 1
8 1
9 5
3 7
3 8
4 9
4 9
3 9
9 3
4 9
9 5
3 5
3 8
3 5
8 1


In [56]:
with open('1NN_rf1.txt', 'w') as f:
    f.write(str(error_rate_1NN))
!cat 1NN_rf1.txt

0.0377777777778

Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [57]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [58]:
from sklearn.metrics import accuracy_score
error_rate_rf = 1 - accuracy_score(y_test, rf.predict(X_test))

In [59]:
with open('1NN_rf2.txt', 'w') as f:
    f.write(str(error_rate_rf))
!cat 1NN_rf2.txt

0.0688888888889